In [1]:
##Load metadata from file

import ast

d = {}
nary = {}
with open("metadata_trial4.txt", 'r') as f:
    for line in f:
        d = (ast.literal_eval(line))
        nary.update({d['id'] : d})

In [4]:
#Filter terms that have children and aspect

filnary = {}
kilnary = {}
nochild = {}

for key, value in nary.items():
    if value.get('children') != None:
        filnary = ({'id' : value['id'], 'name' : value['name'], 'aspect' : value['aspect'], 
                    'children' : value['children']})
        kilnary.update({key : filnary})
    else:
        filnary = ({'id' : value['id'], 'name' : value['name'], 'aspect' : value['aspect']})
        nochild.update({key : filnary})

In [5]:
#import owlready2

import types
from owlready2 import *
onto = get_ontology("http://test.org/onto.owl")

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [6]:
#define core classes

with onto:
    class therapeutic_agent(Thing):
        namespace = onto
    class biological_process(Thing):
        namespace = onto
    class molecular_function(Thing):
        namespace = onto
    class drug(Thing):
        namespace = onto

In [7]:
#classmaker function to make FTC classes

def classmaker(term, parentclass):
                classnameA = 'GOA' + str(term)
                classnameP = 'GOP' + str(term)
                with onto:
                    classname = types.new_class(classnameA, (parentclass,))
                    classname = types.new_class(classnameP, (parentclass,))
                return 0

In [8]:
#Using classmaker to make FTC classes

for key, value in kilnary.items():
    if value['aspect'] == 'biological_process':
        for item in value['children']:
            if item['relation'] == 'positively_regulates':
                classmaker(str(item['id'][3:]), therapeutic_agent)
            if item['relation'] == 'negatively_regulates':
                classmaker(str(item['id'][3:]), therapeutic_agent)
            if item['relation'] == 'regulates':
                classmaker(str(item['id'][3:]), therapeutic_agent)
    if value['aspect'] == 'molecular_function':
        for item in value['children']:
            if item['relation'] == 'positively_regulates':
                classmaker(str(item['id'][3:]), therapeutic_agent)
            if item['relation'] == 'negatively_regulates':
                classmaker(str(item['id'][3:]), therapeutic_agent)
            if item['relation'] == 'regulates':
                classmaker(str(item['id'][3:]), therapeutic_agent)

In [9]:
kilnarynobpmf = kilnary
bpkeys = []
mfkeys = []
for key, value in nary.items():
    if value['name'] == 'biological_process':
        bpkeys.append(key)
    if value['name'] == 'molecular_function':
        mfkeys.append(key)

for item in bpkeys:
    del kilnarynobpmf[item]
for item in mfkeys:
    del kilnarynobpmf[item]

In [12]:
for key, value in nary.items():
    if value.get('children') == None:
        value['children'] = []

In [13]:
#Making BP and MF classes recursively

def hierarchy_builder(ID, parent):
    with onto:
        classname = types.new_class(ID, (parent,))
    parent = getattr(onto, ID)
    for item in nary[ID]['children']:
        if item['relation'] == 'is_a':
            hierarchy_builder(item['id'], parent)
    return 0

for key, value in kilnarynobpmf.items():
    if value['aspect'] == 'biological_process':
        hierarchy_builder(key, biological_process)
    if value['aspect'] == 'molecular_function':
        hierarchy_builder(key, molecular_function)

TypeError: a __bases__ item causes an inheritance cycle

In [9]:
#Making the 'actual' BP and MF classes

for item in bpkeys:
    hierarchy_builder(item, Thing)
for item in mfkeys:
    hierarchy_builder(item, Thing)
    

KeyError: 'GO:0008150'

In [ ]:
for key, value in kilnary.items():
    if value.get('aspect') == None:
        print('huh')

In [ ]:
estroy_entity(Drug)

In [ ]:
for key, value in nary.items():
    if value['name'] == 'biological_process':
        print(key)

In [ ]:
bpkeys = []
mfkeys = []
for key, value in nary.items():
    if value['name'] == 'biological_process':
        bpkeys.append(key)
    if value['name'] == 'molecular_function':
        mfkeys.append(key)n

In [16]:
onto.save(file = "testowl2", format = "rdfxml")